# HW 3, problem 3

Neural networks for digits data

In [1]:
# NN to recognize hand-written digits using the MNIST data
using DelimitedFiles
using StatsBase
using Distributions
using LinearAlgebra

# read the MNIST data
const testx = readdlm("testx.csv", ',', Int, '\n')
const testy = readdlm("testy.csv", ',', Int, '\n')
const trainx = readdlm("trainx.csv", ',', Int, '\n')
const trainy = readdlm("trainy.csv", ',', Int, '\n')

const L = 3                 # number of layers including input and output
const sizes = [784, 30, 10] # number of neurons in each layer

# the activation function
@. f(z) = 1/(1 + exp(-z))      # sigmoid activation
@. fprime(z) = f(z) * (1-f(z))

# convert a digit d to a 10-element vector
# e.g. 6 is converted to [0,0,0,0,0,0,1,0,0,0]
function digit2vector(d)
    return vcat( repeat([0],d), 1, repeat([0],9-d) )
end

digit2vector (generic function with 1 method)

### Part a) Backpropagation and Gradient Descent

In [2]:
# a feedforward function that returns the activations
# from each layer and the weighted inputs to each layer
# so that they can be used during backpropagation.
# W,b contain the weights, biases in the network.
# x is the input of a single training example (a vector of length 784).
function feedforward(W, b, x)
    a = [ x, zeros(sizes[2]), zeros(sizes[3]) ]
    z = [ zeros(sizes[1]), zeros(sizes[2]), zeros(sizes[3]) ]

    z[2] = W[1] * x + b[1]
    a[2] = f(z[2])
    z[3] = W[2] * a[2] + b[2]
    a[3] = f(z[3])
    
    return a, z
end

feedforward (generic function with 1 method)

In [3]:
# given an input vector, return the predicted digit
function classify(W, b, x)
    (a, z) = feedforward(W, b, x)
    return argmax(a[3]) - 1
end

classify (generic function with 1 method)

In [4]:
# helper function for backprop().
# this function computes the error for a single training example.
# W contains the weights in the network.
# a contains the activations.
# z contains the weighted inputs.
# y is the correct digit.
# returns δ = the error. the size of δ is [ 784, 30, 10 ]
function compute_error(W, a, z, y)
    δ = [ zeros(sizes[1]), zeros(sizes[2]), zeros(sizes[3]) ]
    # note that δ[1] is junk. we put it there so that the indices make sense.

    # at the output layer L
    δ[3] = -(digit2vector(y) .- a[3]) .* fprime(z[3])

    # for each earlier layer L-1,L-2,..,2 (for the HW, this means only layer 2)
    δ[2] = W[2]' * δ[3] .* fprime(z[2])

    return δ
end

compute_error (generic function with 1 method)

In [5]:
# helper function for backprop(). given the errors δ and the
# activations a for a single training example, this function returns
# the gradient components ∇W and ∇b.
# this function implements the equations BP3 and BP4.
function compute_gradients(δ, a)
    ∇W = [ zeros(sizes[2], sizes[1]),
           zeros(sizes[3], sizes[2]) ]    # ((30x784), (10x30))
    ∇b = [ zeros(sizes[2]), zeros(sizes[3]) ]    # ((30x1), (10x1))
    
    ∇W[1] = δ[2] * a[1]'    # (30x1) x (1x784)
    ∇W[2] = δ[3] * a[2]'    # (10x1) x (1x30)
    ∇b[1] = δ[2]    # 30x1
    ∇b[2] = δ[3]    # 10x1
    
    return ∇W, ∇b
end

compute_gradients (generic function with 1 method)

In [6]:
# backpropagation. returns ∇W and ∇b for a single training example.
function backprop(W, b, x, y)
    (a, z) = feedforward(W, b, x)
    δ = compute_error(W, a, z, y)
    (∇W, ∇b) = compute_gradients(δ, a)
    return ∇W, ∇b
end

backprop (generic function with 1 method)

In [7]:
# gradient descent algorithm.
# W = weights in the network
# b = biases in the network
# batch = the indices of the observations in the batch, i.e. the rows of trainx
# α = step size
# λ = regularization parameter
function GD(W, b, batch; α=0.01, λ=0.01)
    m = length(batch)    # batch size

    # data structure to accumulate the sum over the batch.
    # in the notes and in Ng's article sumW is ΔW and sumb is Δb.
    sumW = [ zeros(sizes[2], sizes[1]),
             zeros(sizes[3], sizes[2]) ]
    sumb = [ zeros(sizes[2]), zeros(sizes[3]) ]

    # for each training example in the batch, use backprop
    # to compute the gradients and add them to the sum
    for i in 1:m
        (∇W, ∇b) = backprop(W, b, trainx[batch[i], :], trainy[batch[i]])
        sumW = sumW + ∇W
        sumb = sumb + ∇b
    end

    # make the update to the weights and biases and take a step
    # of gradient descent. note that we use the average gradient.
    ∇W = (1/m) .* sumW + λ.*W
    ∇b = (1/m) .* sumb
    W = W - α .* ((1/m) .* sumW + λ.*W)
    b = b - α .* ((1/m) .* sumb)

    # return the updated weights and biases. we also return the gradients
    return W, b, ∇W, ∇b
end

GD (generic function with 1 method)

### Part c) Function to calculate the Classification Accuracy

This function calculates the proportion of test samples that are correctly classified after one epoch.

In [8]:
# classify the test data and compute the classification accuracy
function accuracy(W, b) 
    ntest = length(testy)
    yhat = zeros(Int, ntest)
    for i in 1:ntest
        yhat[i] = classify(W, b, testx[i,:])
    end
    return sum(testy .== yhat)/ntest # hit rate
end

accuracy (generic function with 1 method)

### Part b) Train the neural network and Calculate the classification accuracy

In [9]:
# train the neural network using batch gradient descent.
# this is a driver function to repeatedly call GD().
# N = number of observations in the training data.
# m = batch size
# α = learning rate / step size
# λ = regularization parameter
function BGD(N, m, epochs; α=0.01, λ=0.01) 
    # random initialization of the weights and biases
    d = Normal(0, 1)
    W = [ rand(d, sizes[2], sizes[1]),  # layer 1 to 2
          rand(d, sizes[3], sizes[2]) ] # layer 2 to 3
    b = [ rand(d, sizes[2]),   # layer 2
          rand(d, sizes[3]) ]  # layer 3
    ∇W = [ zeros(sizes[2], sizes[1]),  # layer 1 to 2
          zeros(sizes[3], sizes[2]) ] # layer 2 to 3
    ∇b = [ zeros(sizes[2]),   # layer 2
          zeros(sizes[3]) ]   # layer 3
    
    for i in 1:epochs
        remaining = 1:60000
        while length(remaining) > 0
            batch = sample(remaining, m)
            remaining = setdiff(remaining, batch)
            (W, b, ∇W, ∇b) = GD(W, b, batch)
        end
        accuracy_rate = accuracy(W, b)
        println("Epoch number ", i, ", accuracy rate ", accuracy_rate)
    end
    
    return W, b, ∇W, ∇b
end

BGD (generic function with 1 method)

In [10]:
# some tuning parameters
N = length(trainy)
m = 20       # batch size
epochs = 30  # number of complete passes through the training data
α = 0.01     # learning rate / step size
λ = 0.01     # regularization parameter
W, b, ∇W, ∇b = BGD(N, m, epochs, α=α, λ=λ)

Epoch number 1, accuracy rate 0.1356
Epoch number 2, accuracy rate 0.1962
Epoch number 3, accuracy rate 0.2948
Epoch number 4, accuracy rate 0.3953
Epoch number 5, accuracy rate 0.5203
Epoch number 6, accuracy rate 0.6008
Epoch number 7, accuracy rate 0.6691
Epoch number 8, accuracy rate 0.7382
Epoch number 9, accuracy rate 0.7918
Epoch number 10, accuracy rate 0.8473
Epoch number 11, accuracy rate 0.8633
Epoch number 12, accuracy rate 0.8795
Epoch number 13, accuracy rate 0.8881
Epoch number 14, accuracy rate 0.8958
Epoch number 15, accuracy rate 0.8947
Epoch number 16, accuracy rate 0.8997
Epoch number 17, accuracy rate 0.8981
Epoch number 18, accuracy rate 0.8952
Epoch number 19, accuracy rate 0.9013
Epoch number 20, accuracy rate 0.8939
Epoch number 21, accuracy rate 0.8998
Epoch number 22, accuracy rate 0.9003
Epoch number 23, accuracy rate 0.9003
Epoch number 24, accuracy rate 0.9009
Epoch number 25, accuracy rate 0.9023
Epoch number 26, accuracy rate 0.8978
Epoch number 27, accu

([[-0.00011406305338195666 -0.00014226568993077924 … 6.168292841667688e-5 0.0001190171662232156; -2.8463270715875076e-5 7.398299498850874e-5 … -0.00012146465620489647 7.447742497371777e-5; … ; -1.1099753485309021e-6 -2.2351709674146346e-5 … 0.00021809552203799065 -2.3954551635602565e-5; -9.475144900215776e-5 5.073437780367948e-5 … -0.00024569744038471943 4.579729857995711e-5], [0.024123171853695714 -0.03891809844912488 … -0.09023286459127625 -0.02395364231598385; 0.0033083406294131474 0.015172651472068293 … -0.06476309513965042 0.5207648866277951; … ; -0.3598756572017 0.03891651285489906 … -0.05576568824347911 -0.12743351467699388; 0.015028526586535567 -0.08517013129027273 … -0.19299913148131098 -0.060816438903199904]], [[-0.43067155850212036, -0.06583076615395542, -0.1553442973891528, -1.0745673808163534, 1.3336266234338696, -0.3382831935710984, -0.09083842081333818, 0.3339418195791132, 0.9703927158373492, -0.5026437567248545  …  0.5318940568942931, 0.0838206040531174, -1.493877386188

After I train the neural networks using gradient descent with backpropagation, the neural networks achieve an accuracy of roughly 90%, meaning that given a set of input data, the networks will classify the digits correctly 90% of the time. The weights and biases are initialized randomly but through gradient descent with backpropagation, the networks "learn" and modify those matrices accordingly for better classification.